In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
import time
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show


In [ ]:
from background_inputs import background_inputs
from model import model
from solver import solver

# Phase space of $\pi-\sigma$ model

In [ ]:
from scipy.integrate import odeint

m2 = 1
H = 1
rho = 1
k = 1

def system(y, N):
    pi, dpi, sigma, dsigma = y
    dydN = [dpi, -rho/H*(sigma + dsigma) - 3*dpi - k**2/H**2/np.exp(2*N)*pi, dsigma, rho/H*dpi - (k**2/H**2/np.exp(2*N) + m2/H**2)*sigma - 3*dsigma]
    return dydN


N = np.linspace(-3, 10, 1000)
y0 = [1/np.sqrt(2*k)*np.exp(N[0]), k/np.sqrt(2*k), 1/np.sqrt(2*k)*np.exp(N[0]), k/np.sqrt(2*k)]

sol = odeint(system, y0, N)


In [ ]:
plt.plot(N, sol[:, 0], label='$\pi$')
plt.plot(N, sol[:, 2], label='$\sigma$')
plt.legend(loc = 'best', frameon = False)
plt.xlabel('$N$', fontsize = 15)
plt.grid()

In [ ]:
plt.plot(sol[:, 0], sol[:, 2])
plt.xlabel("$\pi$", fontsize = 15)
plt.ylabel("$\sigma$", fontsize = 15)

In [ ]:
plt.plot(N, sol[:, 0]**2)

## Time-evolution of various correlators

In [ ]:
ref = 0.49973742837658897 #Sigma_{pi pi} for decoupled fields

In [ ]:
N_load = np.linspace(-5, 10, 500)
H_load = np.ones(500)
rho_load = 5*np.ones(500)
m2_load = 0.5*np.ones(500)

Omega_load = 0
Vabc_load = 0
Rasbs_load = 0
Rabcs_load = 0
Rasbs_c_load = 0

background = background_inputs(N_load, H_load, rho_load, Omega_load, m2_load, Vabc_load, Rasbs_load, Rabcs_load, Rasbs_c_load)
interpolated = background.output()

In [ ]:
%matplotlib
Nspan = np.linspace(-5, 10, 500)
Nfield = 2
Rtol, Atol = [1e-3, 1e-5, 1e-3], [1e-100, 1e-100, 1e-6]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)


N_exit = 0
k = mdl.k_mode(N_exit)
print("k = {}".format(k))

DeltaN = 5 # number of efolds before horizon crossing
Ni, Nf, Nsample = N_exit - DeltaN, 10, 1000 # sets initial and final efolds for transport equation integration
N = np.linspace(Ni, Nf, Nsample)
s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)


start_time = time.time()
rescaling = k**3/2/np.pi**2
ref = 0.49973742837658897*rescaling #Sigma_{pi pi} for decoupled fields
SigmaAB = s.SigmaAB_solution(k = k, part = "Re")*rescaling #to have the dimensionless power spectrum
print("The power spectra computation took", time.time() - start_time, "sec to run")


#Plotting the power spectra
plt.semilogy(N, np.absolute(SigmaAB[0, 0])/ref, label = "$XX = \pi\pi$")
plt.semilogy(N, np.absolute(SigmaAB[1, 1])/ref*np.exp(3*N), label = "$XX = \sigma\sigma$")
#plt.semilogy(N, np.absolute(SigmaAB[0, 1]), label = "$XX = \zeta\cal{F}$")
#plt.semilogy(N, np.absolute(SigmaAB[2, 2]), label = "$XX = p_\pi p_\pi$")
#plt.semilogy(N, np.absolute(SigmaAB[3, 3]), label = "$XX = p_\sigma p_\sigma$")
#plt.semilogy(N, np.absolute(SigmaAB[2, 3]), label = "$XX = \pi_\zeta\pi_{\cal{F}}$")
#plt.semilogy(N, np.absolute(SigmaAB[0, 2]), label = "$XX = \zeta\pi_\zeta$")
#plt.semilogy(N, np.absolute(SigmaAB[1, 3]), label = "$XX = \cal{F}\pi_{\cal{F}}$")

#plt.semilogy(N, np.absolute(SigmaAB[2, 2] - 3*rho_load[0]**2*SigmaAB[1, 1]), label = "$XX = \dot{\pi} \dot{\pi}$")


#meff2 = m2_load[0] + rho_load[0]**2
#nu = np.sqrt(9/4 - meff2/H_load[0]**2)
#plt.semilogy(N, np.exp(-(3-2*nu)*N), ls = "--", color = "k", label = "$1/a^{3-2\\nu}$")
#plt.semilogy(N, np.exp(-3*N), ls = "--", color = "k", label = "$1/a^3$")

plt.axvline(x = N_exit, ls = "--", color = "grey")
plt.xlabel("$N$", fontsize = 15)
#plt.ylim(1e-8, 1e5)
plt.title("massive case, $m^2/H^2 = {}, \\rho/H = {}$".format(m2_load[0], rho_load[0]), fontsize = 15)
plt.ylabel("Re$(\Sigma^{XX})/\cal{P}_{\pi}$", fontsize = 15)
plt.grid(True)
plt.legend(frameon = False)
plt.show()

## Parameter space scan

In [ ]:
N_load = np.linspace(-5, 10, 500)
H_load = np.ones(500)

Omega_load = 0
Vabc_load = 0
Rasbs_load = 0
Rabcs_load = 0
Rasbs_c_load = 0


In [ ]:
n = 10
m2 = np.linspace(0, 10, n)
rho2 = np.linspace(0, 10, n)

pipi = []

for i in range(n):
    print("i=",i)
    A = []
    for j in range(n):
        print("j=", j)
        rho_load = np.sqrt(rho2[i])*np.ones(500)
        m2_load = m2[j]*np.ones(500)
        
        background = background_inputs(N_load, H_load, rho_load, Omega_load, m2_load, Vabc_load, Rasbs_load, Rabcs_load, Rasbs_c_load)
        interpolated = background.output()
        
        Nspan = np.linspace(-5, 10, 500)
        Nfield = 2
        Rtol, Atol = [1e-3, 1e-5, 1e-3], [1e-100, 1e-100, 1e-6]

        mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

        N_exit = 0
        k = mdl.k_mode(N_exit)

        DeltaN = 5 # number of efolds before horizon crossing
        Ni, Nf, Nsample = N_exit - DeltaN, 10, 1000 # sets initial and final efolds for transport equation integration
        N = np.linspace(Ni, Nf, Nsample)
        s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)
        
        rescaling = k**3/2/np.pi**2
        ref = 0.49973742837658897*rescaling #Sigma_{pi pi} for decoupled fields
        SigmaAB = s.SigmaAB_solution(k = k, part = "Re")*rescaling #to have the dimensionless power spectrum
        
        Correlators = SigmaAB/ref
        A.append(Correlators[0, 0])
    pipi.append(A)
        

In [ ]:
%matplotlib
X, Y = meshgrid(m2, rho2) # grid of point
Z = np.transpose(pipi, (2, 0, 1))[-1]


plt.pcolor(X, Y, 1/Z, cmap = cm.RdBu)
clb = plt.colorbar()
plt.clim(0, 1)
plt.title("$\\beta(m^2/H^2, \\rho^2/H^2)$", fontsize = 15)
plt.xlabel("$m^2/H^2$", fontsize = 15)
#ax.set_xticks([-1, 0, 1])
plt.ylabel("$\\rho^2/H^2$", fontsize = 15)
#ax.set_yticks([-1, 0, 1])


plt.show()    

### Plot one-dimensional slices

In [ ]:
#Heavy case
m2 = np.linspace(0, 10, n)
rho2 = np.linspace(0, 10, n)
Z_heavy = []
cs = (1 + rho2/(m2[-1] + rho2))**(-1/2)

for i in range(n):
        Z_heavy.append(pipi[i][-1][-1])

        
plt.plot(rho2, 1/np.asarray(Z_heavy))
plt.plot(rho2, cs)
plt.plot(rho2, np.sqrt(cs/np.sqrt(m2[-1] + rho2)*np.sqrt(m2[-1])))
plt.xlabel("$\\rho^2/H^2$", fontsize = 15)
plt.ylabel("$\\beta(m^2/H^2, \\rho^2/H^2)$", fontsize = 15)
plt.title("$m^2/H^2=10$", fontsize = 15)

# Initial conditions

In [ ]:
from background_inputs import background_inputs
from model import model
from solver import solver

In [ ]:
n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
rho_load = 1e2*np.ones(n_back)
m2_load = 1e-1*np.ones(n_back)

mu_load = 0*np.ones(n_back)
kappa1_load = 1*np.ones(n_back)
kappa2_load = 0*np.ones(n_back)

background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load)
interpolated = background.output()

In [ ]:
#Solve the bispectrum
%matplotlib
Nspan = np.linspace(-10, 10, 500)
Nfield = 2
Rtol, Atol = [1e-3, 1e-3, 1e-3], [1e-6, 1e-3, 1e-180]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

N_exit = 0
kt = mdl.k_mode(N_exit)
k1, k2, k3 = kt, kt, kt
rescaling = k1**3/2/np.pi**2
print("k1 = {}, k2 = {}, k3 = {}".format(k1, k2, k3))

DeltaN = 8 # number of efolds before horizon crossing
Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
N = np.linspace(Ni, Nf, 1000)
s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

start_time = time.time()
f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

In [ ]:
S = 1/(2*np.pi)**4/((f[0][0, 0][-1]*k1**3/2/np.pi**2 + f[1][0, 0][-1]*k2**3/2/np.pi**2 + f[2][0, 0][-1]*k3**3/2/np.pi**2)/3)**2 * (k1*k2*k3)**2 * f[6][0, 0, 0]
fNL = 5/18 * f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2
print("fNL = ", fNL)
print("S = ", S[-1])

In [ ]:
Nfield = 2
N_IC = np.linspace(-10, 10, 1000)
Rtol, Atol = [1e-3, 1e-5, 1e-3], [1e-1, 1e-100, 1e-6]

Binit = np.zeros((2*Nfield, 2*Nfield, 2*Nfield, len(N)))
for i in range(len(N)):
    Ni = np.asarray([N_IC[i]])
    s = solver(Nspan = Ni, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)
    Binit[:, :, :, i] = s.B_init(k1, k2, k3)

In [ ]:
P1 = np.absolute(f[0][0, 0]) * k1**3/2/np.pi**2
P2 = np.absolute(f[1][0, 0]) * k2**3/2/np.pi**2
P3 = np.absolute(f[2][0, 0]) * k3**3/2/np.pi**2

plt.semilogy(N, P1)
plt.semilogy(N, P2)
plt.semilogy(N, P3)

In [ ]:
#Oscillations in varphi
plt.semilogy(N, np.abs(f[6][0, 0, 0]), color = "C0", label = "$\\varphi\\varphi\\varphi$")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 0]), "--", color = "C0", label = "initial conditions")

plt.semilogy(N, np.abs(f[6][0, 0, 2]), color = "C1", label = "$\\varphi p_{\\varphi}$")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 2]), "--", color = "C1", label = "initial conditions")

plt.semilogy(N, np.abs(f[6][0, 2, 2]), color = "C2", label = "$\\varphi p_{\\varphi}p_{\\varphi}$")
#plt.semilogy(N_IC, np.abs(Binit[0, 2, 2]), "--", color = "C2", label = "initial conditions")

plt.semilogy(N, np.abs(f[6][2, 2, 2]), color = "C3", label = "$p_{\\varphi}p_{\\varphi}p_{\\varphi}$")
#plt.semilogy(N_IC, np.abs(Binit[2, 2, 2]), "--", color = "C3", label = "initial conditions")

plt.xlabel("$N$", fontsize = 15)
plt.axvline(x = N_exit, ls = "--", color = "grey", label = "horizon crossing")
plt.legend(frameon = False)
plt.grid(False)

In [ ]:
#Oscillations in cross-terms varphi/sigma
plt.semilogy(N, np.abs(f[6][0, 0, 1]), color = "C0", label = "$\\varphi\\varphi\sigma$")
plt.semilogy(N_IC, np.abs(Binit[0, 0, 1]), "--", color = "C0")

plt.semilogy(N, np.abs(f[6][0, 1, 1]), color = "C1", label = "$\\varphi\sigma\sigma$")
plt.semilogy(N_IC, np.abs(Binit[0, 1, 1]), "--", color = "C1")

plt.semilogy(N, np.abs(f[6][0, 0, 3]), color = "C2", label = "$\\varphi\\varphi p_{\sigma}$")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 3]), "--", color = "C2")

plt.semilogy(N, np.abs(f[6][0, 3, 3]), color = "C3", label = "$\\varphi p_{\sigma}p_{\sigma}$")
plt.semilogy(N_IC, np.abs(Binit[0, 3, 3]), "--", color = "C3")

plt.semilogy(N, np.abs(f[6][2, 1, 1]), color = "C4", label = "$p_{\\varphi}\sigma\sigma$")
plt.semilogy(N_IC, np.abs(Binit[2, 1, 1]), "--", color = "C4")

plt.semilogy(N, np.abs(f[6][2, 2, 1]), color = "C5", label = "$p_{\\varphi}p_{\\varphi}\sigma$")
plt.semilogy(N_IC, np.abs(Binit[2, 2, 1]), "--", color = "C5")

plt.xlabel("$N$", fontsize = 15)
plt.axvline(x = N_exit, ls = "--", color = "grey", label = "horizon crossing")
plt.legend()
plt.grid(False)

# Shape function

In [ ]:
from background_inputs import background_inputs
from model import model
from solver import solver

In [ ]:
n_back = 500 #Number of points for the background

N_load = np.linspace(-5, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
rho_load = 1*np.ones(n_back)
m2_load = 1*np.ones(n_back)

mu_load = 1*np.ones(n_back)
kappa1_load = 0*np.ones(n_back)
kappa2_load = 1*np.ones(n_back)

background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load)
interpolated = background.output()

In [ ]:
start_time = time.time()
Nspan = np.linspace(-10, 10, n_back)
Nfield = 2
Rtol, Atol = [1e-3, 1e-3, 1e-3], [1e-6, 1e-3, 1e-180]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

N_exit = 0
k1 = mdl.k_mode(N_exit)


n = 30
x3 = np.linspace(0+1e-2, 1-1e-2, n) #k3/k1
x2 = np.linspace(0.5+1e-2, 1-1e-2, n) #k2/k1

Shape = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        print("i = {}/{}, j = {}/{}".format(i, n, j, n))
        if x2[j] >= 0.25*x3[i] + 0.75 or x2[j] <= -0.25*x3[i] + 0.75:
            Shape[i, j] = np.log(0)
        else:
            mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)
            N_exit = 0
            k1 = mdl.k_mode(N_exit)
            k2 = k1*x2[i]
            k3 = k1*x3[j]

            DeltaN = 6 # number of efolds before horizon crossing
            Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
            N = np.linspace(Ni, Nf, 1000)
            s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

            N1 = mdl.efold_mode(k1, Nf, N_exit)
            N2 = mdl.efold_mode(k2, Nf, N_exit)
            N3 = mdl.efold_mode(k3, Nf, N_exit)
            print("N1 = {}, N2 = {}, N3 = {}".format(N1, N2, N3))

            start_time = time.time()
            f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
            print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

            S = 1/(2*np.pi)**4/((f[0][0, 0][-1]*k1**3/2/np.pi**2 + f[1][0, 0][-1]*k2**3/2/np.pi**2 + f[2][0, 0][-1]*k3**3/2/np.pi**2)/3)**2 * (k1*k2*k3)**2 * f[6][0, 0, 0]

            Shape[i, j] = S[-1]


#np.save("x3.npy", x3)
#np.save("x2.npy", x2)
#np.save("Shape.npy", Shape)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)
X3, X2 = np.meshgrid(x3, x2)
ax.contourf(X3, X2, Shape)

ax.set_xticks([0, 0.5, 1])
ax.set_yticks([0.5, 0.75, 1])
ax.plot(0.5*x3+0.5, x2, color = "k", ls = "--")
ax.plot(-0.5*x3+0.5, x2, color = "k", ls = "--")

contourf_ = ax.contourf(X3, X2, Shape, n, cmap = "hot")
cbar = fig.colorbar(contourf_)
plt.show()

In [ ]:
%matplotlib
from matplotlib import cm
from matplotlib.ticker import LinearLocator

Z = Shape

Z[Z < -1e5] = 0

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X3, X2, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax.set_zlim([1e-1, 2])

# Comparison with Assassi/Baumann/Green/McAllister

## Power Spectrum and the $d(M)$ function

In [ ]:
from background_inputs import background_inputs
from model import model
from solver import solver

In [ ]:
n_back = 500 #Number of points for the background

N_load = np.linspace(-5, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
m2_load = 1e-1*np.ones(n_back)
rho_load = 1e2*np.ones(n_back)


mu_load = 0*np.ones(n_back)
kappa1_load = 0*np.ones(n_back)
kappa2_load = 0*np.ones(n_back)

background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load)
interpolated = background.output()

In [ ]:
%matplotlib
Nspan = np.linspace(-10, 10, 500)
Nfield = 2
Rtol, Atol = [1e-4, 1e-5, 1e-3], [1e-100, 1e-100, 1e-6]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)


N_exit = 0
k = mdl.k_mode(N_exit)
print("k = {}".format(k))

DeltaN = 8 # number of efolds before horizon crossing
Ni, Nf, Nsample = N_exit - DeltaN, 10, 1000 # sets initial and final efolds for transport equation integration
N = np.linspace(Ni, Nf, Nsample)
s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)


start_time = time.time()
rescaling = k**3/2/np.pi**2
SigmaAB = s.SigmaAB_solution(k = k, part = "Re") * rescaling #to have the dimensionless power spectrum
print("The power spectra computation took", time.time() - start_time, "sec to run")
print("$d(M)$ = ", np.sqrt(SigmaAB[0, 0][-1])*2*np.pi/rho_load[0]**(1/4))
print("$M = m^2/\\rho = $", m2_load[0]/rho_load[0])



#Plotting the power spectra
plt.semilogy(N, np.absolute(SigmaAB[0, 0]), label = "$XX = \\varphi\\varphi$")
plt.semilogy(N, np.absolute(SigmaAB[1, 1]), label = "$XX = \sigma\sigma$")
plt.semilogy(N, np.absolute(SigmaAB[0, 1]), label = "$XX = \\varphi\sigma$")

plt.semilogy(N, np.absolute(SigmaAB[2, 2]), label = "$XX = p_\\varphi p_\\varphi$")
plt.semilogy(N, np.absolute(SigmaAB[3, 3]), label = "$XX = p_\sigma p_\sigma$")
plt.semilogy(N, np.absolute(SigmaAB[2, 3]), label = "$XX = p_\\varphi p_\sigma$")


plt.axvline(x = N_exit, ls = "--", color = "grey", label = "$k\\tau\sim 1$")
#plt.axvline(x = -np.log(H_load[0]/k*np.sqrt(rho_load[0])), ls = "--", color = "black", label = "$k\\tau\sim \sqrt{\\rho}$")

#cs = 1/np.sqrt(1 + rho_load[0]**2/m2_load[0])
#mu_mass = np.sqrt(m2_load[0]/H_load[0]**2 - 9/4)

#plt.axvline(x = -np.log(H_load[0]/k/cs), ls = "--", color = "black", label = "$c_s k\\tau\sim 1$")
#plt.axvline(x = -np.log(H_load[0]/k*mu_mass), ls = "--", color = "darkred", label = "$k\\tau\sim \mu$")

plt.xlabel("$N$", fontsize = 15)
#plt.ylim(1e-5, 1e8)
#plt.title("$m^2/H^2 = 10, \\rho/H = 15$", fontsize = 15)
plt.grid(False)
plt.legend(frameon = False)

In [ ]:
sqP = np.asarray([8.1769, 4.5967, 2.6974, 1.9172, 1.03486, 1.00892, 1.002242, 1.000996, 1.0005595, 1.00035])
M = np.asarray([1e-3, 1e-2, 1e-1, 1, 10, 20, 40, 60, 80, 100])

plt.loglog(M, sqP)
plt.loglog(M, 1/M**(1/4))
plt.loglog(M, 1/M**(1/2))

In [ ]:
from scipy.special import gamma

np.sqrt(2)*2*gamma(5/4)/np.sqrt(np.pi)

In [ ]:
# Scan

n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
rho_load = 100*np.ones(n_back)


mu_load = 0*np.ones(n_back)
kappa1_load = 0*np.ones(n_back)
kappa2_load = 0*np.ones(n_back)


n_points = 10
m2_array = np.logspace(-1, 4, n_points)
M = []
d = []

for i in range(n_points):
    m2_load = m2_array[i]*np.ones(n_back)
    background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load)
    interpolated = background.output()
    
    Nspan = np.linspace(-10, 10, 500)
    Nfield = 2
    Rtol, Atol = [1e-4, 1e-5, 1e-3], [1e-100, 1e-100, 1e-6]

    mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

    N_exit = 0
    k = mdl.k_mode(N_exit)
    #print("k = {}".format(k))

    DeltaN = 8 # number of efolds before horizon crossing
    Ni, Nf, Nsample = N_exit - DeltaN, 10, 1000 # sets initial and final efolds for transport equation integration
    N = np.linspace(Ni, Nf, Nsample)
    s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)


    start_time = time.time()
    rescaling = k**3/2/np.pi**2
    SigmaAB = s.SigmaAB_solution(k = k, part = "Re") * rescaling #to have the dimensionless power spectrum
    print("The power spectra computation took", time.time() - start_time, "sec to run")
    print("$d(M)$ = ", np.sqrt(SigmaAB[0, 0][-1])*2*np.pi/rho_load[0]**(1/4))
    print("$M = m^2/\\rho = $", m2_load[0]/rho_load[0])
    
    M.append(m2_load[0]/rho_load[0])
    d.append(np.sqrt(SigmaAB[0, 0][-1])*2*np.pi/rho_load[0]**(1/4))
    
M = np.asarray(M)
d = np.asarray(d)


In [ ]:
plt.semilogx(M, d)
plt.ylim(0, 1.5)
plt.xlabel("$M/H$", fontsize = 15)
plt.ylabel("$d(M)$", fontsize = 15)
plt.title("$\Delta N = 8, \\frac{\\rho}{H} = 10^2$", fontsize = 15)
#plt.legend(frameon = False)

## Bispectrum $f_1(M)$ function

In [120]:
from background_inputs import background_inputs
from model import model
from solver import solver

In [129]:
n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
m2_load = 1e4*np.ones(n_back)
rho_load = 1e2*(np.tanh((N_load + 6.5)/0.1) + 1)/2#1*np.ones(n_back)


mu_load = 0*np.ones(n_back)
kappa1_load = 1e-50*(np.tanh((-N_load - 6)/0.5) + 1)/2#0*np.ones(n_back)#1*(np.tanh((N_load + 6)/0.5) + 1)/2
kappa2_load = 1*(np.tanh((N_load + 6)/0.5) + 1)/2

mu1_load = 0*np.ones(n_back)
mu2_load = 0*np.ones(n_back)
mu3_load = 0*np.ones(n_back)
mu4_load = 0*np.ones(n_back)
mu5_load = 0*np.ones(n_back)


background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load, mu1_load, mu2_load, mu3_load, mu4_load, mu5_load)
interpolated = background.output()

In [131]:
Nspan = np.linspace(-10, 10, 500)
Nfield = 2
Rtol, Atol = [1e-3, 1e-3, 1e-3], [1e-6, 1e-3, 1e-180]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

N_exit = 0
kt = mdl.k_mode(N_exit)
k1, k2, k3 = kt, kt, kt
rescaling = k1**3/2/np.pi**2
print("k1 = {}, k2 = {}, k3 = {}".format(k1, k2, k3))

DeltaN = 10 # number of efolds before horizon crossing
Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
N = np.linspace(Ni, Nf, 100000)
s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

start_time = time.time()
f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

print("S(k1, k2, k3) = ", 1/(2*np.pi)**4/((f[0][0, 0][-1]*k1**3/2/np.pi**2 + f[1][0, 0][-1]*k2**3/2/np.pi**2 + f[2][0, 0][-1]*k3**3/2/np.pi**2)/3)**2 * (k1*k2*k3)**2 * f[6][0, 0, 0][-1])
print("$f_1(M)$ = ", 5/18*f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2/rho_load[-1])
print("$M = m^2/\\rho = $", m2_load[0]/rho_load[-1])

k1 = 1.0, k2 = 1.0, k3 = 1.0
The spectra/bispectra computation took 443.0072178840637 sec to run
S(k1, k2, k3) =  3.555382879723869e-05
$f_1(M)$ =  3.9504254219154097e-07
$M = m^2/\rho = $ 100.0


In [ ]:
np.sqrt(f[0][0, 0][-1]*kt**3/2/np.pi**2)*2*np.pi/rho_load[-1]**(1/4)

In [128]:
cs = 1/np.sqrt(1 + rho_load[-1]**2/m2_load[0])
S_theor = -1/18*(1-cs**2)*kappa2_load[-1]*H_load[0]/m2_load[0]
print(cs)
print(S_theor)

0.7071067811865475
-2.7777777777777783e-06


In [132]:
%matplotlib
plt.semilogy(N, np.absolute(f[0][0, 0]), label = "$\\varphi\\varphi$")

plt.semilogy(N, np.absolute(f[0][1, 1]), label = "$\sigma\sigma$")
plt.semilogy(N, np.absolute(f[0][0, 1]), label = "$\\varphi\sigma$")

plt.semilogy(N, np.absolute(f[0][2, 2]), label = "$p_\\varphi p_\\varphi$")
plt.semilogy(N, np.absolute(f[0][3, 3]), label = "$p_\sigma p_\sigma$")
plt.semilogy(N, np.absolute(f[0][2, 3]), label = "$p_\\varphi p_\sigma$")

plt.semilogy(N, np.absolute(f[6][0, 0, 0]), label = "$\\varphi\\varphi\\varphi$")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 0]), "--")
plt.semilogy(N, np.absolute(f[6][1, 1, 1]), label = "$\sigma\sigma\sigma$")
#plt.semilogy(N_IC, np.abs(Binit[1, 1, 1]), "--")

cs = 1/np.sqrt(1 + rho_load[-1]**2/m2_load[0])
mu_mass = np.sqrt(m2_load[0]/H_load[0]**2 - 9/4)

plt.axvline(x = -np.log(H_load[0]/kt*np.sqrt(rho_load[-1])), ls = "--", color = "darkblue", label = "$k\\tau\sim \sqrt{\\rho}$")
plt.axvline(x = -np.log(H_load[0]/kt/cs), ls = "--", color = "black", label = "$c_s k\\tau\sim 1$")
plt.axvline(x = -np.log(H_load[0]/kt*mu_mass), ls = "--", color = "darkred", label = "$k\\tau\sim \mu$")

plt.xlabel("$N$", fontsize = 15)
plt.axvline(x = N_exit, ls = "--", color = "grey", label = "horizon crossing")
plt.legend(frameon = False)
plt.grid(False)
plt.ylim(1e-25, 1e10)

Using matplotlib backend: MacOSX


(1e-25, 10000000000.0)

In [ ]:
X = np.asarray([0.01, 0.05, 0.1, 0.5, 1, 5, 10])
plt.semilogx(X, np.asarray([3.079, 3.034, 2.977, 2.52, 2.02, 0.31, 0.048])/3*0.45)

In [ ]:
#Scan 

n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
rho_load = 1e2*(np.tanh((N_load + 6.5)/0.1) + 1)/2


mu_load = 0*np.ones(n_back)
kappa1_load = 1*(np.tanh((N_load + 6)/0.5) + 1)/2
kappa2_load = 0*np.ones(n_back)

mu1_load = 0*np.ones(n_back)
mu2_load = 0*np.ones(n_back)
mu3_load = 0*np.ones(n_back)
mu4_load = 0*np.ones(n_back)
mu5_load = 0*np.ones(n_back)


n_points = 10
m2_array = np.logspace(-1, 4, n_points)
M = []
f1 = []

for i in range(n_points):
    print("i=", i)
    m2_load = m2_array[i]*np.ones(n_back)
    background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load, mu1_load, mu2_load, mu3_load, mu4_load, mu5_load)
    interpolated = background.output()
    
    Nspan = np.linspace(-10, 10, 500)
    Nfield = 2
    Rtol, Atol = [1e-3, 1e-3, 1e-3], [1e-6, 1e-3, 1e-300]

    mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

    N_exit = 0
    kt = mdl.k_mode(N_exit)
    k1, k2, k3 = kt, kt, kt
    rescaling = k1**3/2/np.pi**2
    #print("k1 = {}, k2 = {}, k3 = {}".format(k1, k2, k3))

    DeltaN = 10 # number of efolds before horizon crossing
    Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
    N = np.linspace(Ni, Nf, 1000)
    s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

    start_time = time.time()
    f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
    print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

    print("$f_1(M)$ = ", 5/18*f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2/rho_load[-1])
    print("$M = m^2/\\rho = $", m2_load[0]/rho_load[-1])
    
    M.append(m2_load[0]/rho_load[-1])
    f1.append(5/18*f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2/rho_load[-1])
    
M = np.asarray(M)
f1 = np.asarray(f1)

In [ ]:
plt.semilogx(M, f1)
#plt.loglog(M, 1.5/M)
#plt.ylim(0, 1.5)
plt.title("$\Delta N = 10, \\frac{\\rho}{H} = 10^2$", fontsize = 15)
plt.xlabel("$M/H$", fontsize = 15)
plt.ylabel("$f_1(M)$", fontsize = 15)
#plt.legend(frameon = False)

In [ ]:
plt.loglog(M, f1, "-")
plt.loglog(M, 1/M)

In [ ]:
np.save("f1.npy", f1)
np.save("M_f1.npy", M)

## Bispectrum $f_2(M)$ function

In [12]:
from background_inputs import background_inputs
from model import model
from solver import solver

In [99]:
n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
m2_load = 1e4*np.ones(n_back)
rho_load = 1e2*(np.tanh((N_load + 6.5)/0.1) + 1)/2


mu_load = 1*(np.tanh((N_load + 6)/0.5) + 1)/2
kappa1_load = 0*(np.tanh((-N_load - 6)/0.1) + 1)/2#0*np.ones(n_back)
kappa2_load = 1e-50*(np.tanh((-N_load - 6)/0.1) + 1)/2

mu1_load = 0*np.ones(n_back)
mu2_load = 0*np.ones(n_back)
mu3_load = 0*np.ones(n_back)
mu4_load = 0*np.ones(n_back)
mu5_load = 0*np.ones(n_back)

background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load, mu1_load, mu2_load, mu3_load, mu4_load, mu5_load)
interpolated = background.output()

In [100]:
Nspan = np.linspace(-10, 10, 500)
Nfield = 2
Rtol, Atol = [1e-3, 1e-3, 1e-3], [1e-6, 1e-3, 1e-180]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

N_exit = 0
kt = mdl.k_mode(N_exit)
k1, k2, k3 = kt, kt, kt
rescaling = k1**3/2/np.pi**2
print("k1 = {}, k2 = {}, k3 = {}".format(k1, k2, k3))

DeltaN = 10 # number of efolds before horizon crossing
Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
N = np.linspace(Ni, Nf, 100000)
s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

start_time = time.time()
f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

print("S(k1, k2, k3) = ", 1/(2*np.pi)**4/((f[0][0, 0][-1]*k1**3/2/np.pi**2 + f[1][0, 0][-1]*k2**3/2/np.pi**2 + f[2][0, 0][-1]*k3**3/2/np.pi**2)/3)**2 * (k1*k2*k3)**2 * f[6][0, 0, 0][-1])
print("$f_2(M)$ = ", 5/18*f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2 * 2*np.pi * np.sqrt(f[0][0, 0][-1]) * H_load[0]/mu_load[-1] * (rho_load[-1]/H_load[-1])**(3/4))
print("$M = m^2/\\rho = $", m2_load[0]/rho_load[-1])

k1 = 1.0, k2 = 1.0, k3 = 1.0
The spectra/bispectra computation took 461.981153011322 sec to run
S(k1, k2, k3) =  -1.8576913421476378e-07
$f_2(M)$ =  -3.448835159118267e-05
$M = m^2/\rho = $ 100.0


In [107]:
%matplotlib
plt.semilogy(N, np.absolute(f[0][0, 0]), label = "$\\varphi\\varphi$")

plt.semilogy(N, np.absolute(f[0][1, 1]), label = "$\sigma\sigma$")
plt.semilogy(N, np.absolute(f[0][0, 1]), label = "$\\varphi\sigma$")

plt.semilogy(N, np.absolute(f[0][2, 2]), label = "$p_\\varphi p_\\varphi$")
plt.semilogy(N, np.absolute(f[0][3, 3]), label = "$p_\sigma p_\sigma$")
plt.semilogy(N, np.absolute(f[0][2, 3]), label = "$p_\\varphi p_\sigma$")

plt.semilogy(N, np.absolute(f[6][0, 0, 0]), label = "$\\varphi\\varphi\\varphi$")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 0]), "--")
plt.semilogy(N, np.absolute(f[6][1, 1, 1]), label = "$\sigma\sigma\sigma$")
#plt.semilogy(N_IC, np.abs(Binit[1, 1, 1]), "--")

#cs = 1/np.sqrt(1 + rho_load[-1]**2/m2_load[0])
mu_mass = np.sqrt(m2_load[0]/H_load[0]**2 - 9/4)

plt.axvline(x = -np.log(H_load[0]/kt*np.sqrt(rho_load[-1])), ls = "--", color = "darkblue", label = "$k\\tau\sim \sqrt{\\rho}$")
#plt.axvline(x = -np.log(H_load[0]/kt/cs), ls = "--", color = "black", label = "$c_s k\\tau\sim 1$")
plt.axvline(x = -np.log(H_load[0]/kt*mu_mass), ls = "--", color = "darkred", label = "$k\\tau\sim \mu$")

plt.xlabel("$N$", fontsize = 15)
plt.axvline(x = N_exit, ls = "--", color = "grey", label = "horizon crossing")
plt.legend(frameon = False)
plt.grid(False)
plt.ylim(1e-25, 1e10)

Using matplotlib backend: MacOSX


/var/folders/2k/k567rdsx46v7_3gsyc4f07xm0000gp/T/ipykernel_91010/2357093312.py:17: RuntimeWarning: invalid value encountered in sqrt
  mu_mass = np.sqrt(m2_load[0]/H_load[0]**2 - 9/4)


(1e-25, 10000000000.0)

In [48]:
N_load = np.linspace(-10, 10, 500)
y1 = (np.tanh((N_load + 6)/0.4) + 1)/2
y2 = (np.tanh((-N_load - 6)/0.4) + 1)/2

plt.plot(N_load, y1)
plt.plot(N_load, y2)

In [108]:
#Scan 

n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
rho_load = 1e2*(np.tanh((N_load + 6.5)/0.1) + 1)/2


mu_load = 1*(np.tanh((N_load + 6)/0.5) + 1)/2
kappa1_load = 0*(np.tanh((-N_load - 6)/0.1) + 1)/2#0*np.ones(n_back)
kappa2_load = 1e-50*(np.tanh((-N_load - 6)/0.1) + 1)/2

mu1_load = 0*np.ones(n_back)
mu2_load = 0*np.ones(n_back)
mu3_load = 0*np.ones(n_back)
mu4_load = 0*np.ones(n_back)
mu5_load = 0*np.ones(n_back)


n_points = 20
m2_array = np.logspace(-1, 4, n_points)
M = []
f2 = []

for i in range(n_points):
    print("i=", i)
    m2_load = m2_array[i]*np.ones(n_back)
    background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load, mu1_load, mu2_load, mu3_load, mu4_load, mu5_load)
    interpolated = background.output()
    
    Nspan = np.linspace(-10, 10, 500)
    Nfield = 2
    Rtol, Atol = [1e-3, 1e-3, 1e-3], [1e-6, 1e-3, 1e-180]

    mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

    N_exit = 0
    kt = mdl.k_mode(N_exit)
    k1, k2, k3 = kt, kt, kt
    rescaling = k1**3/2/np.pi**2
    #print("k1 = {}, k2 = {}, k3 = {}".format(k1, k2, k3))

    DeltaN = 10 # number of efolds before horizon crossing
    Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
    N = np.linspace(Ni, Nf, 1000)
    s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

    start_time = time.time()
    f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
    print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

    print("$f_2(M)$ = ", 5/18*f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2 * 2*np.pi * np.sqrt(f[0][0, 0][-1]) * H_load[0]/mu_load[-1] * (rho_load[-1]/H_load[-1])**(3/4))
    print("$M = m^2/\\rho = $", m2_load[0]/rho_load[-1])
    
    M.append(m2_load[0]/rho_load[-1])
    f2.append(5/18*f[6][0, 0, 0][-1]/f[0][0, 0][-1]**2 * 2*np.pi * np.sqrt(f[0][0, 0][-1]) * H_load[0]/mu_load[-1] * (rho_load[-1]/H_load[-1])**(3/4))
    
M = np.asarray(M)
f2 = np.asarray(f2)

i= 0
The spectra/bispectra computation took 440.46343302726746 sec to run
$f_2(M)$ =  1.1875143608172956
$M = m^2/\rho = $ 0.001
i= 1
The spectra/bispectra computation took 440.5423800945282 sec to run
$f_2(M)$ =  1.1854495527159927
$M = m^2/\rho = $ 0.0018329807108324356
i= 2
The spectra/bispectra computation took 441.4333472251892 sec to run
$f_2(M)$ =  1.1816709010937323
$M = m^2/\rho = $ 0.0033598182862837815
i= 3
The spectra/bispectra computation took 441.04975485801697 sec to run
$f_2(M)$ =  1.1747638376665794
$M = m^2/\rho = $ 0.006158482110660264
i= 4
The spectra/bispectra computation took 440.9948060512543 sec to run
$f_2(M)$ =  1.1621679159155052
$M = m^2/\rho = $ 0.011288378916846888
i= 5
The spectra/bispectra computation took 441.00319266319275 sec to run
$f_2(M)$ =  1.1392969212456454
$M = m^2/\rho = $ 0.020691380811147894
i= 6
The spectra/bispectra computation took 440.63638496398926 sec to run
$f_2(M)$ =  1.0980988240139398
$M = m^2/\rho = $ 0.0379269019073225
i= 7
The s

In [118]:
plt.loglog(M, np.absolute(f2), ".")
plt.loglog(M, 1/M**(9/4))
#plt.ylim(0, 1.5)
plt.title("$\Delta N = 10, \\frac{\\rho}{H} = 10^2$", fontsize = 15)
plt.xlabel("$M/H$", fontsize = 15)
plt.ylabel("$f_2(M)$", fontsize = 15)
#plt.legend(frameon = False)

Text(0, 0.5, '$f_2(M)$')

In [110]:
np.save("f2.npy", f2)
np.save("M_f2.npy", M)

## Bispectrum $f_3(M)$ function

# Compute field-space geometry

In [ ]:
from gravipy.tensorial import * # import GraviPy package
from sympy import init_printing
import inspect
init_printing()

In [ ]:
# define some symbolic variables
phi, sigma, Lambda = symbols('\phi, \sigma, \Lambda')
# create a coordinate four-vector object instantiating 
# the Coordinates class
x = Coordinates('\chi', [phi, sigma])
# define a matrix of a metric tensor components
Metric = Matrix([[1 + sigma/Lambda, -sigma/Lambda], [-sigma/Lambda, 1]])
# create a metric tensor object instantiating the MetricTensor class
g = MetricTensor('g', x, Metric)

In [ ]:
print([cls.__name__ for cls in vars()['Tensor'].__subclasses__()])

In [ ]:
Ga = Christoffel('Ga', g)
Ga(1, 2, 1)

In [ ]:
Ri = Ricci('Ri', g)
Ri(All, All)

In [ ]:
Ri.scalar()

In [ ]:
g(All, All)

# Massive test field in dS with cubic interactions with the Transport Approach

In [2]:
from background_inputs import background_inputs
from model import model
from solver import solver

In [3]:
n_back = 500 #Number of points for the background

N_load = np.linspace(-10, 10, n_back)
H_load = np.ones(n_back) #Hubble scale set to unity
m2_load = 1*np.ones(n_back)
rho_load = 0*np.ones(n_back)

mu1_load = 0*np.ones(n_back)
mu2_load = 1*np.ones(n_back)#(np.tanh((N_load + 6)/0.75) + 1)/2#(np.tanh((N_load + 4)/0.1) + 1)/2
mu3_load = 0*np.ones(n_back)
mu4_load = 0*np.ones(n_back)
mu5_load = 0*np.ones(n_back)#(np.tanh((N_load + 6)/0.1) + 1)/2


mu_load = 0*np.ones(n_back)
kappa1_load = 0*np.ones(n_back)
kappa2_load = 0*np.ones(n_back)

background = background_inputs(N_load, H_load, rho_load, mu_load, m2_load, kappa1_load, kappa2_load, mu1_load, mu2_load, mu3_load, mu4_load, mu5_load)
interpolated = background.output()

In [4]:
Nspan = np.linspace(-10, 10, 500)
Nfield = 2
Rtol, Atol = [1e-3, 1e-3, 1e-4], [1e-6, 1e-3, 1e-180]

mdl = model(N = Nspan, Nfield = Nfield, interpolated = interpolated)

N_exit = 0
kt = mdl.k_mode(N_exit)
k1, k2, k3 = kt, kt, kt
rescaling = k1**3/2/np.pi**2
print("k1 = {}, k2 = {}, k3 = {}".format(k1, k2, k3))

DeltaN = 5 # number of efolds before horizon crossing
Ni, Nf = N_exit - DeltaN, 10 # sets initial and final efolds for transport equation integration
N = np.linspace(Ni, Nf, 100000)
s = solver(Nspan = N, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)

start_time = time.time()
f = s.f_solution(k1 = k1, k2 = k2, k3 = k3)
print("The spectra/bispectra computation took", time.time() - start_time, "sec to run")

print("S(k1, k2, k3) = ", 1/(2*np.pi)**4/((f[0][0, 0][-1]*k1**3/2/np.pi**2 + f[1][0, 0][-1]*k2**3/2/np.pi**2 + f[2][0, 0][-1]*k3**3/2/np.pi**2)/3)**2 * (k1*k2*k3)**2 * f[6][0, 0, 0][-1])

k1 = 1.0, k2 = 1.0, k3 = 1.0
The spectra/bispectra computation took 4.801689863204956 sec to run
S(k1, k2, k3) =  16.479956879821344


In [5]:
%matplotlib
plt.semilogy(N, np.absolute(f[0][0, 0]), label = "$\sigma\sigma$", color = "C4")
plt.semilogy(N, np.absolute(f[0][0, 2]), label = "$\sigma p_\sigma$", color = "C5")
plt.semilogy(N, np.absolute(f[0][2, 2]), label = "$p_\sigma p_\sigma$", color = "C6")

plt.semilogy(N, np.absolute(f[6][0, 0, 0]), label = "$\sigma\sigma\sigma$", color = "C0")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 0]), ls = "--", color = "C0")
plt.semilogy(N, np.absolute(f[6][0, 0, 2]), label = "$\sigma\sigma p_\sigma$", color = "C1")
#plt.semilogy(N_IC, np.abs(Binit[0, 0, 2]), ls = "--", color = "C1")
plt.semilogy(N, np.absolute(f[6][0, 2, 2]), label = "$\sigma p_\sigma p_\sigma$", color = "C2")
#plt.semilogy(N_IC, np.abs(Binit[0, 2, 2]), ls = "--", color = "C2")
plt.semilogy(N, np.absolute(f[6][2, 2, 2]), label = "$p_\sigma p_\sigma p_\sigma$", color = "C3")
#plt.semilogy(N_IC, np.abs(Binit[2, 2, 2]), ls = "--", color = "C3")

#plt.semilogy(N_IC, 1e2*np.exp(-3*N_IC))

"""Light field"""
#nu_mass = np.sqrt(9/4 - m2_load[0]/H_load[0]**2)
#plt.semilogy(N, np.exp(-(3 - 2*nu_mass)*N), "--", color = "black", label = "$1/a^{3-2\\nu}$")


"""Heavy field"""
#mu_mass = np.sqrt(m2_load[0]/H_load[0]**2 - 9/4)
#plt.axvline(x = -np.log(H_load[0]/kt*mu_mass), ls = "--", color = "darkred", label = "$k\\tau\sim \mu$")
#plt.semilogy(N, np.exp(-3*N), "--", color = "black", label = "$1/a^{3}$")

plt.title("$m^2 = H^2$, $\mu_2 = 1$, $N_0 = -6$, $\delta N = 0.75$, $\Delta N = 10$", fontsize = 15)
plt.ylabel("$S(k, k, k) = 298.18$", fontsize = 15)
#plt.title("$m^2 = 10H^2$", fontsize = 15)
plt.xlabel("$N$", fontsize = 15)
plt.axvline(x = N_exit, ls = "--", color = "grey", label = "horizon crossing")
plt.legend(frameon = False)
plt.grid(False)

Using matplotlib backend: MacOSX


In [ ]:
"""Oscillations for the heavy field"""

#plt.semilogy(N_IC, np.abs(Binit[0, 0, 0]), ls = "--")
#plt.semilogy(N, np.absolute(f[0][0, 0])*np.exp(3*N), label = "$\sigma\sigma$")
plt.semilogy(N, np.absolute(f[6][0, 2, 2])*np.exp(3*N), label = "$\sigma\sigma\sigma$")
plt.xlabel("$N$", fontsize = 15)
plt.axvline(x = N_exit, ls = "--", color = "grey", label = "horizon crossing")
plt.legend(frameon = False)
plt.grid(False)


In [ ]:
"""Initial conditions"""

Nfield = 2
N_IC = np.linspace(-10, 10, 1000)
Rtol, Atol = [1e-3, 1e-5, 1e-3], [1e-1, 1e-100, 1e-6]

Binit = np.zeros((2*Nfield, 2*Nfield, 2*Nfield, len(N_IC)))
for i in range(len(N_IC)):
    Ni = np.asarray([N_IC[i]])
    s = solver(Nspan = Ni, Nfield = Nfield, interpolated = interpolated, Rtol = Rtol, Atol = Atol)
    Binit[:, :, :, i] = s.B_init(k1, k2, k3)

In [ ]:
x = np.linspace(-10, 0, 500)

x0 = 5
deltax = 0.1
plt.plot(x, (np.tanh((x + x0)/deltax) + 1)/2, label = "$\delta N = 0.1, N_0 = -5$")

x0 = 5.5
deltax = 0.5
plt.plot(x, (np.tanh((x + x0)/deltax) + 1)/2, label = "$\delta N = 0.5, N_0 = -5.5$")

x0 = 6
deltax = 0.75
plt.plot(x, (np.tanh((x + x0)/deltax) + 1)/2, label = "$\delta N = 0.75, N_0 = -6$")

plt.legend(frameon = False)
plt.xlabel("$N$", fontsize = 15)
plt.ylabel("$\mu(N) = \mu\\left[\\tanh\\frac{N + N_0}{\delta N} +1\\right]/2$", fontsize = 15)